In [17]:
import pandas as pd

In [48]:
df=pd.read_csv(r"C:\Users\Soham Indulkar\OneDrive\Documents\CS DS\BI_Project\Data_Reconciliation_Project\Data\Raw_data.csv")

In [49]:
df.head(2)

,Transaction_ID,User_Name,Age,Country,Product_Category,Purchase_Amount,Payment_Method,Transaction_Date
0,1,Ava Hall,63,Mexico,Clothing,780.69,Debit Card,2023-04-14
1,2,Sophia Hall,59,India,Beauty,738.56,PayPal,2023-07-30


In [50]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction_ID    50000 non-null  int64  
 1   User_Name         50000 non-null  object 
 2   Age               50000 non-null  int64  
 3   Country           50000 non-null  object 
 4   Product_Category  50000 non-null  object 
 5   Purchase_Amount   50000 non-null  float64
 6   Payment_Method    50000 non-null  object 
 7   Transaction_Date  50000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 3.1+ MB
None


In [51]:
print(df.columns)

Index(['Transaction_ID', 'User_Name', 'Age', 'Country', 'Product_Category',
       'Purchase_Amount', 'Payment_Method', 'Transaction_Date'],
      dtype='object')


In [52]:

sales_df = df.copy()

# Save sales data
sales_df.to_csv(r"C:\Users\Soham Indulkar\OneDrive\Documents\CS DS\BI_Project\Data_Reconciliation_Project\Data\sales_data.csv", index=False)


#Creating missing values

In [53]:
# Create finance system data
finance_df = df.copy()

# 1. Remove 
finance_df = finance_df.sample(frac=0.9, random_state=42)

#modify
finance_df.iloc[:10, finance_df.columns.get_loc('Purchase_Amount')] *= 0.95

# Save finance data
finance_df.to_csv(r"C:\Users\Soham Indulkar\OneDrive\Documents\CS DS\BI_Project\Data_Reconciliation_Project\Data\finance_data.csv", index=False)


In [54]:
print(sales_df.shape)



(50000, 8)


In [55]:
print(finance_df.shape)

(45000, 8)


In [56]:
sales_df=pd.read_csv(r"C:\Users\Soham Indulkar\OneDrive\Documents\CS DS\BI_Project\Data_Reconciliation_Project\Data\finance_data.csv")
finance_df=pd.read_csv(r"C:\Users\Soham Indulkar\OneDrive\Documents\CS DS\BI_Project\Data_Reconciliation_Project\Data\sales_data.csv")

In [57]:
sales_df.drop_duplicates(inplace=True)
finance_df.drop_duplicates(inplace=True)


In [58]:
sales_df['Transaction_ID'] = sales_df['Transaction_ID'].astype(str)
finance_df['Transaction_ID'] = finance_df['Transaction_ID'].astype(str)

In [59]:
sales_df.isna().sum()


Transaction_ID      0
User_Name           0
Age                 0
Country             0
Product_Category    0
Purchase_Amount     0
Payment_Method      0
Transaction_Date    0
dtype: int64

In [60]:
finance_df.isna().sum()

Transaction_ID      0
User_Name           0
Age                 0
Country             0
Product_Category    0
Purchase_Amount     0
Payment_Method      0
Transaction_Date    0
dtype: int64

In [61]:
sales_df['Purchase_Amount'] = sales_df['Purchase_Amount'].fillna(0)

In [62]:
finance_df['Purchase_Amount']=finance_df['Purchase_Amount'].fillna(0)

In [63]:
sales_df = sales_df[['Transaction_ID', 'Transaction_Date', 'Product_Category', 'Purchase_Amount']]
finance_df = finance_df[['Transaction_ID', 'Purchase_Amount']]

In [70]:
# missing records
missing_ids = sales_df['Transaction_ID'].sample(20, random_state=1)
finance_df = finance_df[~finance_df['Transaction_ID'].isin(missing_ids)]


In [77]:
recon_df = sales_df.merge(
    finance_df,
    on='Transaction_ID',
    how='left',
    indicator=True
)

missing_orders = recon_df[recon_df['_merge'] == 'left_only']

amount_mismatch = recon_df[
    (recon_df['_merge'] == 'both') &
    (recon_df['Purchase_Amount_x'] != recon_df['Purchase_Amount_y'])
]


In [78]:
print("Total Sales Records:", len(sales_df))
print("Missing Orders:", len(missing_orders))
print("Amount Mismatches:", len(amount_mismatch))


Total Sales Records: 45000
Missing Orders: 20
Amount Mismatches: 10


In [79]:
# Overall MIS Summary
mis_summary = recon_df.groupby('Product_Category').agg(
    Total_Sales=('Purchase_Amount_x', 'sum'),
    Total_Billed=('Purchase_Amount_y', 'sum'),
    Transaction_Count=('Transaction_ID', 'count')
).reset_index()


In [81]:
mis_summary.to_csv(r"C:\Users\Soham Indulkar\OneDrive\Documents\CS DS\BI_Project\Data_Reconciliation_Project\Output\is_summary.csv", index=False)
missing_orders.to_csv(r"C:\Users\Soham Indulkar\OneDrive\Documents\CS DS\BI_Project\Data_Reconciliation_Project\Output\missing_orders.csv", index=False)
amount_mismatch.to_csv(r"C:\Users\Soham Indulkar\OneDrive\Documents\CS DS\BI_Project\Data_Reconciliation_Project\Output\amount_mismatch.csv", index=False)